In [1]:
with open("requirements.txt", "w") as f:
    f.write("kfp==1.8.9\n")
    
!pip install -r requirements.txt  --upgrade --user

     |████████████████████████████████| 296 kB 324 kB/s eta 0:00:01
     |████████████████████████████████| 12.7 MB 1.1 MB/s eta 0:00:01
  Created wheel for kfp: filename=kfp-1.8.9-py3-none-any.whl size=409653 sha256=ab4f148519ef31189260ca86a5a4529d84824e6b8a2b8a7ae6481521d0fd1cd1
  Stored in directory: /home/jovyan/.cache/pip/wheels/74/45/05/323f17840bbf9963e873b7a1e75caba3d5ab927c390c98c740
Successfully built kfp
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
import kfp
from kfp import dsl


def echo1_op(text1):
  return dsl.ContainerOp(
      name='echo1',
      image='library/bash:4.4.23',
      command=['sh', '-c'],
      arguments=['echo "$0"', text1])


def echo2_op(text2):
  return dsl.ContainerOp(
      name='echo2',
      image='library/bash:4.4.23',
      command=['sh', '-c'],
      arguments=['echo "$0"', text2])


@dsl.pipeline(
    name='Execution order pipeline',
    description='A pipeline to demonstrate execution order management.'
)
def execution_order_pipeline(text1='message 1', text2='message 2'):
  """A two step pipeline with an explicitly defined execution order."""
  step1_task = echo1_op(text1)
  step1_task.set_cpu_request("1").set_cpu_limit("1")
  step2_task = echo2_op(text2)
  step2_task.set_cpu_request("1").set_cpu_limit("1")
  step2_task.after(step1_task)

    


In [5]:
# generate workflow artifacts in .zip format
kfp.compiler.Compiler().compile(execution_order_pipeline, 'helloworld-0.zip')